In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from selenium import webdriver  
from selenium.webdriver.common.by import By

In [ ]:
# 2020以下の年はall pageがない

In [80]:
conference = 'cvpr'
year = 2020

conference_url = f"https://openaccess.thecvf.com/CVPR{year}?day=all"
chromedriver_path = '/usr/bin/chromedriver'
data_path = Path("../papers_info")

In [81]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(
    options=options
)

In [82]:
driver.get(conference_url)

In [83]:
title_element_list = driver.find_elements(By.CLASS_NAME, 'ptitle')

titles = []
absts = []
paper_links = []

for element in title_element_list:
    paper_links.append(element.find_element(By.TAG_NAME, 'a').get_attribute('href'))

for i,element in enumerate(title_element_list):
    driver.get(paper_links[i])
    # if Not Found page is shown, skip
    if driver.find_element(By.TAG_NAME, 'body').text[:9] == 'Not Found':
        continue

    title = driver.find_element(By.ID, 'papertitle').text
    abst = driver.find_element(By.ID, 'abstract').text
    titles.append(title)
    absts.append(abst)

In [84]:
#driver.current_url
#driver.find_element(By.TAG_NAME, 'h1').text == 'Not Found'
#driver.find_element(By.TAG_NAME, 'body').text[:9]

In [85]:
print(f"{len(title_element_list)} papers are found.")
print(f"{len(titles)} papers are downloaded.")

0 papers are found.
0 papers are downloaded.


In [86]:
driver.quit()

In [87]:
cvpr = pd.DataFrame.from_dict({
    'year': year,
    'title': titles, 
    'abstract': absts,
    'conference': conference,
})

In [88]:
mask = np.array([len(a) >= 200 for a in cvpr.abstract])
cvpr = cvpr[mask].reset_index(drop=True)
print(f'Removing {np.sum(~mask)} submissions with abstract length below 200 characters.')

TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
cvpr.to_pickle(data_path / f'{conference}_{year}.pickle.zip')